# 00 — Project setup & mental model

This notebook explains **how to run notebooks** and the **mental model** of the project.

The pipeline is:

`OHLCV → Indicators → Universe filters → Ranking → Signals → Risk sizing → Reporting → Backtesting`

Everything is based on a single canonical data structure:

- `pandas.DataFrame` with **DatetimeIndex**
- columns are a **MultiIndex (field, ticker)**, e.g. `("Close","AAPL")`

In [2]:
# If running from repo root and editable install is not done:
# pip install -e ".[dev]"

import pandas as pd
pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 50)

> Tip: many modules assume the benchmark **SPY** is present (for Relative Strength).
> When using real tickers, include SPY:
>
> `tickers = ["AAPL","MSFT","NVDA","SPY"]`

In [3]:
from swing_screener.data.market_data import fetch_ohlcv, MarketDataConfig

tickers = ["AAPL","MSFT","NVDA","SPY"]
ohlcv = fetch_ohlcv(tickers, MarketDataConfig(start="2023-01-01"))

ohlcv.head()

Open                                           High                                            Low              \
                  AAPL        MSFT       NVDA         SPY        AAPL        MSFT       NVDA         SPY        AAPL        MSFT   
Date                                                                                                                               
2023-01-03  128.343772  237.403914  14.836149  369.481541  128.954553  240.011566  14.981005  371.461746  122.324579  231.856538   
2023-01-04  125.004155  226.856104  14.552433  368.337652  126.747853  227.432323  14.838146  370.933080  123.221057  220.683688   
2023-01-05  125.240583  221.894695  14.476506  366.934233  125.871071  222.236528  14.549433  367.049580  122.905811  216.581721   
2023-01-06  124.137247  217.792793  14.459527  367.789759  128.353629  220.488340  14.994991  374.172576  123.033890  214.228029   
2023-01-09  128.530950  221.162221  15.268716  375.249150  131.427258  225.840379  16.039943  378.450180  127.959568  221.123161   

                                        Close                                        Volume                                  
                 NVDA         SPY        AAPL        MSFT       NVDA         SPY       AAPL      MSFT       NVDA        SPY  
Date                                                                                                                         
2023-01-03  14.081904  363.194858  123.211205  233.985641  14.300685  366.069061  112117500  25740000  401277000   74850700  
2023-01-04  14.226759  365.280836  124.482033  223.750366  14.734250  368.895203   89113600  50623400  431324000   85934100  
2023-01-05  14.133849  364.088896  123.161942  217.118881  14.250732  364.684875   80962700  39585600  389168000   76970500  
2023-01-06  14.019967  364.713727  127.693588  219.677719  14.844142  373.047882   87754700  43613600  405044000  104189600  
2023-01-09  15.125858  372.653751  128.215698  221.816574  15.612371  372.836365   70790800  27369800  504231000   73978100

Look at the columns: they are MultiIndex `(field, ticker)`.

Most modules accept this `ohlcv` and output **per-ticker tables** (index = ticker).


## Definitions (core data model)
- OHLCV data is a Pandas DataFrame indexed by date.
- Columns use a MultiIndex: (field, ticker), e.g. (Close, AAPL).
- Many modules assume the benchmark (default SPY) is present for Relative Strength.



## OHLCV fields
- Open: first traded price of the day.
- High: highest traded price of the day.
- Low: lowest traded price of the day.
- Close: last traded price of the day (used for most indicators).
- Volume: total shares traded (used for liquidity checks / context).

Why we need them:
- Trend and momentum use Close prices.
- ATR uses High/Low/Close to measure daily range.
- Backtests need Open/High/Low/Close to model entry/exit and stop logic.



# Setup using the Universe Provider
This section mirrors the setup flow, but loads tickers from a universe (package CSV or local file).


Pipeline:
`OHLCV → Indicators → Universe filters → Ranking → Signals → Risk sizing → Reporting → Backtesting`

Canonical data structure:
- `pandas.DataFrame` with **DatetimeIndex**
- **MultiIndex columns (field, ticker)** e.g. `("Close","AAPL")`

New: tickers can come from a **Universe Provider** (package CSV or file).

In [4]:
import pandas as pd
pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 80)

## Load tickers from a packaged universe

In [5]:
from swing_screener.data.universe import load_universe_from_package, UniverseConfig
from swing_screener.data.market_data import fetch_ohlcv, MarketDataConfig

tickers = load_universe_from_package("mega", UniverseConfig(max_tickers=25))
tickers[:12], len(tickers)

(['TICKER',
  'SPY',
  'QQQ',
  'DIA',
  'IWM',
  'XLK',
  'XLF',
  'XLE',
  'XLV',
  'XLY',
  'XLI',
  'XLP'],
 25)

## Fetch OHLCV using those tickers

In [6]:
ohlcv = fetch_ohlcv(tickers, MarketDataConfig(start="2023-01-01"))
ohlcv.head()

Open                                                                                                                     \
           TICKER         SPY         QQQ         DIA         IWM        XLK        XLF        XLE         XLV        XLY        XLI   
Date                                                                                                                                   
2023-01-03    NaN  369.481572  263.892028  315.409418  169.426262  61.536336  32.827763  39.263493  129.361093  63.328396  94.336107   
2023-01-04    NaN  368.337652  261.917623  315.542236  168.164663  60.621389  33.037403  37.560508  129.608485  63.162690  94.297914   
2023-01-05    NaN  366.934202  259.363719  314.033618  168.135808  60.161466  33.113643  38.057406  128.371557  63.040826  94.288358   
2023-01-06    NaN  367.789699  258.685968  315.181668  168.819531  59.814085  33.199409  39.317697  128.761711  62.719146  94.746760   
2023-01-09    NaN  375.249089  266.033408  320.409658  172.392385  61.575472  33.828323  40.148858  129.218379  65.151285  96.609034   

                                                                                                                                        \
                  XLP        XLU        XLB       XLRE        AAPL        MSFT       NVDA       AMZN        META      GOOGL       GOOG   
Date                                                                                                                                     
2023-01-03  68.797698  32.235548  36.686353  33.565455  128.343780  237.403930  14.836148  85.459999  121.968023  88.918160  89.161279   
2023-01-04  68.696253  32.290262  36.771216  33.718725  125.004162  226.856104  14.552433  86.550003  126.496383  89.672453  90.332487   
2023-01-05  68.640893  32.189949  36.643927  33.917060  125.240591  221.894726  14.476509  85.330002  125.255047  86.814054  87.414364   
2023-01-06  68.954483  32.094205  36.973963  33.457261  124.137217  217.792778  14.459525  83.029999  128.075353  86.139150  86.709660   
2023-01-09  69.849166  32.367781  38.204543  34.313753  128.530950  221.162251  15.268715  87.459999  130.250194  87.697379  88.531010   

                                                 High                                                                                   \
                  TSLA       BRK-B         JPM TICKER         SPY         QQQ         DIA         IWM        XLK        XLF        XLE   
Date                                                                                                                                     
2023-01-03  118.470001  310.070007  125.039647    NaN  371.461777  265.365462  316.747240  170.764891  61.942437  33.056460  39.462251   
2023-01-04  109.110001  312.000000  125.733074    NaN  370.933080  262.713275  316.946512  170.138882  60.851350  33.466213  38.341982   
2023-01-05  110.510002  313.570007  126.354468    NaN  367.049549  259.530692  314.204400  168.299522  60.318034  33.113643  39.028605   
2023-01-06  103.000000  315.000000  126.792247    NaN  374.172514  265.159253  319.868881  171.458230  61.301486  33.818802  40.035930   
2023-01-09  118.959999  319.019989  129.092790    NaN  378.450118  270.414440  322.041643  173.461351  62.862269  34.018906  40.212098   

                                                                                          ...      Close                                   \
                   XLV        XLY        XLI        XLP        XLU        XLB       XLRE  ...        XLI        XLP        XLU        XLB   
Date                                                                                      ...                                               
2023-01-03  129.713130  63.508740  94.488912  68.825375  32.344978  36.926812  33.862974  ...  93.992302  68.502548  32.144360  36.587341   
2023-01-04  129.941504  63.952290  95.128777  69.194313  32.727972  37.327570  34.494072  ...  94.823174  68.770035  32.436165  37.223843   
2023-01-0

Most modules accept `ohlcv` and output **per-ticker tables** (index = ticker).


## Glossary (tickers and metrics)
- SPY: ETF tracking the S&P 500. We use it as the benchmark for Relative Strength (RS).
- SMA: Simple Moving Average. Used for trend filters and pullback signals.
- ATR: Average True Range. Measures typical daily range; used for stop distance and sizing.
- RS: Relative Strength = momentum vs benchmark (e.g., stock 6m return minus SPY 6m return).
- R (R-multiple): Risk unit where 1R = entry - stop. Used in backtests and trade management.
